In [1]:
import sys
# Append root path 
sys.path.append("../")
sys.path.append("../lmmnn")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

if tf.test.gpu_device_name() != '/device:GPU:0':
    print('WARNING: GPU device not found.')
else:
    print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

from model.mixed_effects import *
from utils.fe_models import get_model
from utils.evaluation import *
from utils.utils import *
from data.preprocessing import dataset_preprocessing
from utils.training_functions import *

# from vis.utils.utils import apply_modifications
# # helper function
def update_layer_activation(model, activation, index=-1):
    model.layers[index].activation = activation
    return model

from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Reshape, Embedding, Concatenate
from tensorflow.keras.activations import sigmoid

from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import roc_auc_score as auroc
from sklearn.metrics import f1_score as f1
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from tensorflow_addons.metrics import F1Score

from scipy import stats
import pickle
import yaml
import time
import gc

RS = 555

SUCCESS: Found GPU: /device:GPU:0


In [2]:
class MCMCSamplingCallback(tf.keras.callbacks.Callback):
    def __init__(self,
                 num_mcmc_samples=1,
                 step_size=0.01,
                 perc_burnin=0.1,
                 num_burnin_steps=0,
                 warm_restart=None):
        super().__init__()

        self.num_mcmc_samples = tf.constant(num_mcmc_samples)
        self.perc_burnin = perc_burnin
        self.num_burnin_steps = num_burnin_steps
        self.warm_restart = warm_restart
        self.step_size = tf.Variable(step_size,trainable=False)
        self.step_sizes = []


    def on_train_begin(self, logs=None):
        self.mcmc_kernel = tfp.mcmc.NoUTurnSampler(
            target_log_prob_fn=self.model.target_log_prob_fn,
            step_size=self.step_size)

#         self.get_mcmc_kernel = lambda step_size: tfp.mcmc.NoUTurnSampler(
#             target_log_prob_fn=self.model.target_log_prob_fn,
#             step_size=step_size)

        # self.mcmc_kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
        #     inner_kernel=tfp.mcmc.NoUTurnSampler(
        #         target_log_prob_fn=self.model.target_log_prob_fn,
        #         step_size=self.step_size),
        #     num_adaptation_steps=500,
        #     target_accept_prob=0.651)

    def on_epoch_begin(self, epoch, logs=None):
        if epoch==0:
            self.model.all_samples.extend(([[state[num] for state in self.model.current_state] for num in range(1)]))

            self.model.mean_samples = [tf.reduce_mean([sample[q] for sample in self.model.all_samples[round(epoch*(self.perc_burnin)):]], axis=0) for q in
                                       range(len(self.model.qs))]
        if self.model.fe_pretraining:
            if self.model.fe_converged:
                self.run_sampling(epoch)
            else:
                self.model.acceptance_rates.append(-1)
        else:
            self.run_sampling(epoch)

    def run_sampling(self,epoch):
        self.model.fX.assign(self.model.fe_model(self.model.X, training=False))

        if self.model.embed_x:
            self.model.X_embedded.assign(self.model.X_embed_model(self.model.X, training=False))

        if self.model.embed_z:
            for q_num in range(len(self.model.qs)):
                self.model.Z_embedded[q_num].assign(self.model.Z_embed_models[q_num](self.model.Z[:,q_num], training=False))

                ## Find initial step size
        # if self.model.previous_kernel_results.log_accept_ratio == -np.inf:
        # if len(self.model.acceptance_rates)>0 and self.model.acceptance_rates[-1]<0.5:
#         if len(self.model.acceptance_rates)>0 and self.model.acceptance_rates[-1]<0.0001:
            # self.mcmc_kernel.parameters["step_size"] = self.mcmc_kernel.parameters["step_size"]/2
            # self.model.previous_kernel_results["new_step_size"] = self.model.previous_kernel_results.step_size/2
            # setattr(self.model.previous_kernel_results, "new_step_size", self.model.previous_kernel_results.step_size/2)
#             self.step_size.assign(self.step_size/2)
#             print(f"Adapt step size to {float(self.step_size)}")


        if self.warm_restart!=None and epoch>0:
            ## Warm restart
            # if self.model.previous_kernel_results.log_accept_ratio == -np.inf:
                # restart = True
            # else:
                # restart = False
            # else:
            restart = ((epoch + 1) % self.warm_restart) == 0 and epoch != 0

            if restart:
                print("\n Warm restart to unstuck the chain")
                if self.model.embed_z and self.model.embed_x:
                    self.model.current_state = self.model.data_model(self.model.fX, self.model.X_embedded, self.model.Z_embedded).sample(1, seed=self.model.RS)[:-1]
                elif self.model.embed_z and not self.model.embed_x:
                    self.model.current_state = self.model.data_model(self.model.fX, self.model.X, self.model.Z_embedded).sample(1, seed=self.model.RS)[:-1]
                elif not self.model.embed_z and self.model.embed_x:
                    self.model.current_state = self.model.data_model(self.model.fX, self.model.X_embedded, self.model.Z).sample(1, seed=self.model.RS)[:-1]
                else:
                    self.model.current_state = self.model.data_model(self.model.fX, self.model.X, self.model.Z).sample(1, seed=self.model.RS)[:-1]

        print("\n Start sampling for epoch {} of training".format(epoch + 1))
        start = time.time()
        new_state, self.model.previous_kernel_results = self.get_mcmc_samples(self.model.current_state,
                                                                              tf.constant(self.num_mcmc_samples),
                                                                              None
                                                                                               )
        # self.model.divide_constants.assign(
        #     list(1/np.mean(self.model.data_model._stddev_z,axis=1))+[1.])
        # self.model.divide_constants.assign(
        #     list((lambda x: 1+(x-x.mean()))(np.array(1+tf.math.softmax(1/len(self.model.qs)+0.5*tf.math.softmax(np.abs([np.mean(i) for i in self.model.previous_kernel_results.grads_target_log_prob]))))))+[1.])
        # print(np.round(self.model.divide_constants,2))
        try:
            log_accept_ratio = self.model.previous_kernel_results.log_accept_ratio
        except:
            log_accept_ratio = self.model.previous_kernel_results.inner_results.log_accept_ratio
        acceptance_rate = tf.math.exp(tf.minimum(log_accept_ratio, 0.))

        self.step_sizes.append(float(self.step_size))

        end = time.time()


        self.model.current_state = [tf.identity(i) for i in new_state]
        # Todo: Append all current states
        self.model.acceptance_rates.append(acceptance_rate)
        # self.model.all_samples.append(
        #     [tf.math.reduce_mean(self.model.current_state[q_num], axis=0) for q_num in range(len(self.model.qs))])
        self.model.all_samples.extend(([[state[num] for state in self.model.current_state] for num in range(self.num_mcmc_samples)]))

        self.model.mean_samples = [tf.reduce_mean([sample[q] for sample in self.model.all_samples[round(epoch*(self.perc_burnin)):]], axis=0) for q in
                                   range(len(self.model.qs))]

        self.model.e_step_times.append(round(end - start, 2))

    # def on_epoch_end(self, epoch, logs=None):
        for q_num in range(len(self.model.qs)):
                self.model.data_model.trainable_variables[q_num].assign(
                    tf.math.reduce_std(self.model.current_state[q_num][-1],axis=0))

        self.model.stds.append([tf.identity(i) for i in self.model.data_model._stddev_z])

    @tf.function(reduce_retracing=True)  # autograph=False, jit_compile=True, reduce_retracing=True)
    def get_mcmc_samples(self, current_state, num_mcmc_samples=tf.constant(1), previous_kernel_results=None):
        samples, _, previous_kernel_results = tfp.mcmc.sample_chain(
            kernel=self.mcmc_kernel, num_results=num_mcmc_samples,
            current_state=[state[-1] for state in current_state],
            num_burnin_steps=self.num_burnin_steps,
            trace_fn=None, previous_kernel_results=previous_kernel_results,
            return_final_kernel_results=True, seed=self.model.RS)
        #     current_state=[sample[-1] for sample in samples]

        return samples, previous_kernel_results

#### Download and save data from Pargent et al. by running "data/download_pargent2022_datasets.py before running this notebook

In [3]:
mode="cv"
hct=10
test_ratio=None
val_ratio=None
folds=5
results = {}
dataset_names = ["churn", "kdd_internet_usage", "Amazon_employee_access", "Click_prediction_small", "adult", "KDDCup09_upselling", "kick", "open_payments", "road-safety-drivers-sex", "porto-seguro"]


loss_use = lambda: tf.keras.losses.BinaryCrossentropy

target= "binary"
batch_size=512
epochs = 500
early_stopping = 20
model_name = "AutoGluon"
embed_dims_method = "AutoGluon"


results = {}

#######################################

for dataset_name in dataset_names:
    print(f"Start training procedure for {dataset_name}")
    data_path = f"{mode}_RS{RS}_hct{hct}"
    if mode == "cv":
        data_path += f"_{folds}folds"
    elif mode == "train_test":
        data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
    elif mode == "train_val_test":
        data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"

    # If no data_dict exists, run preprocessing, else load data_dict
    if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
        dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
    with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
            data_dict = pickle.load(handle)

    z_cols = data_dict["z_cols"]
    results[dataset_name] = {}
    for fold_num in range(folds):
        results[dataset_name][fold_num] = {}

        print(f"Fold no. {fold_num}")
        results[dataset_name][fold_num]["histories"] = {}
        results[dataset_name][fold_num]["predictions"] = {}
        results[dataset_name][fold_num]["times"] = {}
        results[dataset_name][fold_num]["other_info"] = {}
        for step_size in [0.1,0.01,0.001]:
            if step_size==0.1:
                save_path = f"../results/{dataset_name}/{data_path}/fold_{fold_num}/step_size_"+str(step_size)
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
            else:
                save_path = f"../results/{dataset_name}/{data_path}/fold_{fold_num}/step_size__"+str(step_size)
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                
        
            z_ohe_encoded_train = data_dict[f"z_ohe_encoded_train_{fold_num}"] 
            z_ohe_encoded_val = data_dict[f"z_ohe_encoded_val_{fold_num}"] 
            z_ohe_encoded_test = data_dict[f"z_ohe_encoded_test_{fold_num}"] 

            z_target_encoded_train = data_dict[f"z_target_encoded_train_{fold_num}"] 
            z_target_encoded_val = data_dict[f"z_target_encoded_val_{fold_num}"] 
            z_target_encoded_test = data_dict[f"z_target_encoded_test_{fold_num}"] 

            target_encoding_time = data_dict[f"target_encoding_time_{fold_num}"]
            ohe_encoding_time = data_dict[f"ohe_encoding_time_{fold_num}"]

            x_cols = data_dict[f"X_train_{fold_num}"].columns
            X_train = data_dict[f"X_train_{fold_num}"]
            Z_train = data_dict[f"Z_train_{fold_num}"]
            y_train = data_dict[f"y_train_{fold_num}"]

            X_val = data_dict[f"X_val_{fold_num}"]
            Z_val = data_dict[f"Z_val_{fold_num}"]
            y_val = data_dict[f"y_val_{fold_num}"]

            X_test = data_dict[f"X_test_{fold_num}"]
            Z_test = data_dict[f"Z_test_{fold_num}"]
            y_test = data_dict[f"y_test_{fold_num}"]

            if not os.path.exists(f"{save_path}/results_RS{RS}_{dataset_name}_iter{fold_num}.pickle"):

                tf.random.set_seed(RS+fold_num)
                np.random.seed(RS+fold_num)

                qs = np.max([tf.reduce_max(Z_train, axis=0),tf.reduce_max(Z_val, axis=0),tf.reduce_max(Z_test, axis=0)],axis=0)+1

                X_train = tf.convert_to_tensor(X_train)
                Z_train = tf.convert_to_tensor(Z_train,dtype=tf.int32)
                y_train = tf.convert_to_tensor(y_train)

                X_val = tf.convert_to_tensor(X_val)
                Z_val = tf.convert_to_tensor(Z_val,dtype=tf.int32)
                y_val = tf.convert_to_tensor(y_val)

                X_test = tf.convert_to_tensor(X_test)
                Z_test = tf.convert_to_tensor(Z_test,dtype=tf.int32)
                y_test = tf.convert_to_tensor(y_test)

                if target == "categorical":
                    n_classes = np.unique(y_train).shape[0]
                elif target=="binary":
                    n_classes = 1

                y_train = tf.one_hot(tf.cast(y_train,tf.int32),n_classes)
                y_val = tf.one_hot(tf.cast(y_val,tf.int32),n_classes)
                y_test = tf.one_hot(tf.cast(y_test,tf.int32),n_classes)

                ##### GMENN #####
                d = X_train.shape[1] # columns
                n = X_train.shape[0] # rows
                num_outputs = n_classes
                perc_numeric = d/(d+Z_train.shape[1])

    #             qs = np.max([tf.reduce_max(Z_train, axis=0),tf.reduce_max(Z_val, axis=0),tf.reduce_max(Z_test, axis=0)],axis=0)+1

                set_seed(RS)

                fe_model, optimizer = get_model(model_name=model_name, input_size=X_train.shape[1], 
                                                  output_size=num_outputs, 
                                                  target=target, 
                                                  perc_numeric=perc_numeric, RS=RS)

                initial_stds = np.ones([len(qs),num_outputs]).astype(float).tolist()

                me_model = MixedEffectsNetwork(X_train, Z_train, y_train, fe_model, 
                                               target=target, qs=qs,
                                               initial_stds=initial_stds,
                                              fe_loss_weight=1.,
                                               mode="intercepts",
                                               early_stopping_fe=early_stopping,
                                              )    

                me_model.compile(
                    loss_class_me = loss_use()(),
                    loss_class_fe = loss_use()(),
                #     metric_class_me = tf.keras.metrics.AUC(multi_label=True, name="auc_me"),
                #     metric_class_fe = tf.keras.metrics.AUC(multi_label=True, name="auc_fe"),
                    optimizer=optimizer
                )

                mcmc = MCMCSamplingCallback(num_mcmc_samples=1,
                                            perc_burnin=0.7,
                                            warm_restart=None,
                                            num_burnin_steps=1,
                                            step_size = step_size#initial_step_size,
                                       )


                print_metric = PrintMetrics(X_train, Z_train, y_train, X_val, Z_val, y_val)

                start = time.time()
                history = me_model.fit([X_train,Z_train], y_train,
                             callbacks=[mcmc,
                                        print_metric,
                                        tf.keras.callbacks.EarlyStopping(monitor="me_auc_val", patience=early_stopping, mode="max")],
                             epochs=epochs,
                             validation_data=[[X_val,Z_val],y_val],
                            batch_size=batch_size)

                end = time.time()
                fit_time_gmenn = round(end-start,2)

                y_train_pred_gmenn, y_train_pred_gmenn_fe = me_model([X_train,Z_train])
                y_val_pred_gmenn, y_val_pred_gmenn_fe = me_model([X_val,Z_val])
                y_test_pred_gmenn, y_test_pred_gmenn_fe = me_model([X_test,Z_test])    


                ###### Prepare NN Training ######



                ##### Document Results #####

                results[dataset_name][fold_num]["histories"]["GMENN"+str(step_size)] = history.history

                results[dataset_name][fold_num]["predictions"]["GMENN"+str(step_size)] = [y_train_pred_gmenn, y_val_pred_gmenn, y_test_pred_gmenn]
                
                results[dataset_name][fold_num]["times"]["GMENN"+str(step_size)] = fit_time_gmenn

                results[dataset_name][fold_num]["other_info"]["GMENN"+str(step_size)] = {
                        "_stddev_z": np.array([i.numpy() for i in me_model.data_model._stddev_z]),
                        "acceptance_rates": np.array(me_model.acceptance_rates),
                        "random_effects": me_model.mean_samples,
                        "all_samples": me_model.all_samples,
                        "stds": me_model.stds
                    }

                with open(f"{save_path}//results_RS{RS}_{dataset_name}_iter{fold_num}.pickle", 'wb') as handle:
                    pickle.dump(results[dataset_name][fold_num], handle, protocol=pickle.HIGHEST_PROTOCOL)


                del X_train, X_val, X_test, y_train, y_val, y_test

                gc.collect()
            else:
                print(f"Load results for dataset {dataset_name}, iteration={fold_num}")
                with open(f"{save_path}/results_RS{RS}_{dataset_name}_iter{fold_num}.pickle", 'rb') as handle:
                    res = pickle.load(handle)
                results[dataset_name][fold_num]["histories"]["GMENN"+str(step_size)] = res["histories"]["GMENN"+str(step_size)]
                results[dataset_name][fold_num]["predictions"]["GMENN"+str(step_size)] = res["predictions"]["GMENN"+str(step_size)]
                results[dataset_name][fold_num]["times"]["GMENN"+str(step_size)] = res["times"]["GMENN"+str(step_size)]
                results[dataset_name][fold_num]["other_info"]["GMENN"+str(step_size)] = res["other_info"]["GMENN"+str(step_size)]


Start training procedure for churn
Fold no. 0
Load results for dataset churn, iteration=0
Random seed set as 555


2024-01-19 11:38:34.785155: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory



 Start sampling for epoch 1 of training
Epoch 1/500
6/6 [==============================] - 4s 374ms/step - me_loss: 0.6391 - me_loss_val: 0.6465 - fe_loss: 0.6217 - fe_loss_val: 0.6217 - me_auc: 0.5129 - me_auc_val: 0.5268 - fe_auc: 0.5794 - fe_auc_val: 0.5913 - stds: 1.0968 - acceptance_rate: 1.0000

 Start sampling for epoch 2 of training
Epoch 2/500
6/6 [==============================] - 1s 110ms/step - me_loss: 0.3721 - me_loss_val: 0.4086 - fe_loss: 0.5918 - fe_loss_val: 0.5948 - me_auc: 0.6607 - me_auc_val: 0.6630 - fe_auc: 0.6358 - fe_auc_val: 0.6328 - stds: 1.1073 - acceptance_rate: 1.0000

 Start sampling for epoch 3 of training
Epoch 3/500
6/6 [==============================] - 1s 206ms/step - me_loss: 0.3543 - me_loss_val: 0.3958 - fe_loss: 0.5612 - fe_loss_val: 0.5669 - me_auc: 0.7010 - me_auc_val: 0.6977 - fe_auc: 0.6534 - fe_auc_val: 0.6461 - stds: 0.9424 - acceptance_rate: 1.0000

 Start sampling for epoch 4 of training
Epoch 4/500
6/6 [==============================] -


 Start sampling for epoch 28 of training
Epoch 28/500
6/6 [==============================] - 1s 235ms/step - me_loss: 0.2959 - me_loss_val: 0.3511 - fe_loss: 0.3278 - fe_loss_val: 0.3805 - me_auc: 0.8789 - me_auc_val: 0.8255 - fe_auc: 0.7977 - fe_auc_val: 0.7105 - stds: 0.6482 - acceptance_rate: 0.9991

 Start sampling for epoch 29 of training
Epoch 29/500
6/6 [==============================] - 1s 113ms/step - me_loss: 0.2943 - me_loss_val: 0.3500 - fe_loss: 0.3257 - fe_loss_val: 0.3795 - me_auc: 0.8817 - me_auc_val: 0.8291 - fe_auc: 0.8026 - fe_auc_val: 0.7145 - stds: 0.6278 - acceptance_rate: 0.9972

 Start sampling for epoch 30 of training
Epoch 30/500
6/6 [==============================] - 0s 92ms/step - me_loss: 0.2909 - me_loss_val: 0.3477 - fe_loss: 0.3223 - fe_loss_val: 0.3776 - me_auc: 0.8861 - me_auc_val: 0.8303 - fe_auc: 0.8098 - fe_auc_val: 0.7152 - stds: 0.5982 - acceptance_rate: 0.9949

 Start sampling for epoch 31 of training
Epoch 31/500
6/6 [==========================


 Start sampling for epoch 55 of training
Epoch 55/500
6/6 [==============================] - 1s 111ms/step - me_loss: 0.2204 - me_loss_val: 0.3063 - fe_loss: 0.2483 - fe_loss_val: 0.3439 - me_auc: 0.9602 - me_auc_val: 0.8661 - fe_auc: 0.9243 - fe_auc_val: 0.7481 - stds: 0.5405 - acceptance_rate: 0.9992

 Start sampling for epoch 56 of training
Epoch 56/500
6/6 [==============================] - 1s 145ms/step - me_loss: 0.2168 - me_loss_val: 0.3044 - fe_loss: 0.2445 - fe_loss_val: 0.3419 - me_auc: 0.9629 - me_auc_val: 0.8673 - fe_auc: 0.9298 - fe_auc_val: 0.7492 - stds: 0.5796 - acceptance_rate: 0.9983

 Start sampling for epoch 57 of training
Epoch 57/500
6/6 [==============================] - 0s 27ms/step - me_loss: 0.2158 - me_loss_val: 0.3051 - fe_loss: 0.2430 - fe_loss_val: 0.3427 - me_auc: 0.9641 - me_auc_val: 0.8672 - fe_auc: 0.9348 - fe_auc_val: 0.7502 - stds: 0.6502 - acceptance_rate: 0.9983

 Start sampling for epoch 58 of training
Epoch 58/500
6/6 [==========================

...


 Start sampling for epoch 105 of training
Epoch 105/500
275/275 [==============================] - 2s 8ms/step - me_loss: 0.5043 - me_loss_val: 0.5362 - fe_loss: 0.5620 - fe_loss_val: 0.5647 - me_auc: 0.7784 - me_auc_val: 0.7346 - fe_auc: 0.7042 - fe_auc_val: 0.7021 - stds: 0.7823 - acceptance_rate: 0.9980

 Start sampling for epoch 106 of training
Epoch 106/500
275/275 [==============================] - 2s 8ms/step - me_loss: 0.5039 - me_loss_val: 0.5358 - fe_loss: 0.5623 - fe_loss_val: 0.5650 - me_auc: 0.7791 - me_auc_val: 0.7353 - fe_auc: 0.7045 - fe_auc_val: 0.7018 - stds: 0.7888 - acceptance_rate: 0.9839
Random seed set as 555

 Start sampling for epoch 1 of training
Epoch 1/500
275/275 [==============================] - 4s 9ms/step - me_loss: 0.6275 - me_loss_val: 0.6439 - fe_loss: 0.5993 - fe_loss_val: 0.6015 - me_auc: 0.6467 - me_auc_val: 0.6268 - fe_auc: 0.6295 - fe_auc_val: 0.6292 - stds: 1.1339 - acceptance_rate: 1.0000

 Start sampling for epoch 2 of training
Epoch 2/500
2


 Start sampling for epoch 26 of training
Epoch 26/500
275/275 [==============================] - 2s 8ms/step - me_loss: 0.5011 - me_loss_val: 0.5383 - fe_loss: 0.5718 - fe_loss_val: 0.5733 - me_auc: 0.7815 - me_auc_val: 0.7317 - fe_auc: 0.6992 - fe_auc_val: 0.6987 - stds: 0.8631 - acceptance_rate: 0.9998

 Start sampling for epoch 27 of training
Epoch 27/500
275/275 [==============================] - 3s 9ms/step - me_loss: 0.5009 - me_loss_val: 0.5386 - fe_loss: 0.5738 - fe_loss_val: 0.5756 - me_auc: 0.7831 - me_auc_val: 0.7305 - fe_auc: 0.7008 - fe_auc_val: 0.6997 - stds: 0.8951 - acceptance_rate: 1.0000

 Start sampling for epoch 28 of training
Epoch 28/500
275/275 [==============================] - 2s 6ms/step - me_loss: 0.5012 - me_loss_val: 0.5390 - fe_loss: 0.5716 - fe_loss_val: 0.5734 - me_auc: 0.7815 - me_auc_val: 0.7302 - fe_auc: 0.7000 - fe_auc_val: 0.6986 - stds: 0.8905 - acceptance_rate: 0.9956

 Start sampling for epoch 29 of training
Epoch 29/500
275/275 [===============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



698/698 [==============================] - 5s 7ms/step - me_loss: 0.1338 - me_loss_val: 0.1607 - fe_loss: 0.1352 - fe_loss_val: 0.1620 - me_auc: 0.7877 - me_auc_val: 0.5946 - fe_auc: 0.7799 - fe_auc_val: 0.5932 - stds: 0.2007 - acceptance_rate: 1.0000

 Start sampling for epoch 12 of training
Epoch 12/500
698/698 [==============================] - 5s 7ms/step - me_loss: 0.1313 - me_loss_val: 0.1623 - fe_loss: 0.1327 - fe_loss_val: 0.1636 - me_auc: 0.8022 - me_auc_val: 0.5910 - fe_auc: 0.7935 - fe_auc_val: 0.5881 - stds: 0.1874 - acceptance_rate: 1.0000

 Start sampling for epoch 13 of training
Epoch 13/500
698/698 [==============================] - 5s 7ms/step - me_loss: 0.1287 - me_loss_val: 0.1641 - fe_loss: 0.1300 - fe_loss_val: 0.1654 - me_auc: 0.8162 - me_auc_val: 0.5837 - fe_auc: 0.8074 - fe_auc_val: 0.5806 - stds: 0.1913 - acceptance_rate: 1.0000

 Start sampling for epoch 14 of training
Epoch 14/500
698/698 [==============================] - 4s 6ms/step - me_loss: 0.1257 - me_l

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Evaluation

### Performance

In [4]:
models = ["GMENN"+str(step_size) for step_size in [0.1,0.01,0.001]]
results_perf = {dataset_name: {num: {model: {}  for model in models} for num in range(folds)} for dataset_name in dataset_names}
for dataset_name in dataset_names:
    try:
        with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
            data_dict = pickle.load(handle)        
    except:
        print(f"dataset {dataset_name} not found") 
    for num in range(folds):
#         print(num)
        n_classes=1
        y_test = tf.one_hot(data_dict[f"y_test_{num}"],n_classes)
        for model in models:
            try:
                y_pred = np.array(results[dataset_name][num]["predictions"][model][2]).ravel()

                results_perf[dataset_name][num][model] = get_metrics(y_test,y_pred,target)
                results_perf[dataset_name][num][model]["Time"] = results[dataset_name][num]["times"][model]
            except:
                print(f"Set nan for {dataset_name}, {num}")
                results_perf[dataset_name][num][model] = {"Accuracy": np.nan,
                                                          "AUROC": np.nan,
                                                          "F1": np.nan,
                                                          "Time": np.nan}


In [5]:
metric = "AUROC"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

round_mean_at = 2
round_std_at = 3

for dataset_name in dataset_names:
    dataset_models = list(results_perf[dataset_name][0].keys())
    use_df = pd.DataFrame([pd.DataFrame(results_perf[dataset_name][fold_num]).loc[metric,models] for fold_num in results_perf[dataset_name].keys()],index=results_perf[dataset_name].keys())
    
    df_mean = pd.DataFrame(use_df.mean(axis=0).round(round_mean_at).astype(str) + " (" + use_df.std(axis=0).round(round_std_at).astype(str) + ")").transpose()
    model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}
    dataset_res_dict[dataset_name] = model_dict
    
    best_models[dataset_name] = use_df.columns[use_df.mean(axis=0).argmax()]

    t_test_res = np.array([stats.ttest_rel(use_df[best_models[dataset_name]].values, use_df[model].values)[1] if model in dataset_models else 0 for model in models]).round(3)
    t_test_res[np.isnan(t_test_res)] = 1.
    t_test_results[dataset_name] = t_test_res
    
res_df = pd.DataFrame(dataset_res_dict).transpose()
    
def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_results[dataset_name][i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

# res_df.style.apply(negative_bold)
res_df

,GMENN0.1,GMENN0.01,GMENN0.001
churn,0.88 (0.016),0.88 (0.018),0.87 (0.021)
kdd_internet_usage,0.91 (0.016),0.94 (0.003),0.94 (0.003)
Amazon_employee_access,0.52 (0.034),0.84 (0.012),0.83 (0.011)
Click_prediction_small,0.63 (0.037),0.64 (0.016),0.65 (0.016)
adult,0.88 (0.006),0.91 (0.002),0.91 (0.003)
KDDCup09_upselling,0.72 (0.02),0.78 (0.013),0.78 (0.019)
kick,0.67 (0.009),0.74 (0.01),0.74 (0.011)
open_payments,0.51 (0.039),0.93 (0.004),0.92 (0.011)
road-safety-drivers-sex,0.61 (0.01),0.73 (0.003),0.73 (0.004)
porto-seguro,0.53 (0.004),0.56 (0.004),0.56 (0.003)


In [6]:
res_df["GMENN0.1"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN0.01"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN0.001"].apply(lambda x: float(x.split(" ")[0])).values

(array([0.88, 0.91, 0.52, 0.63, 0.88, 0.72, 0.67, 0.51, 0.61, 0.53]),
 array([0.88, 0.94, 0.84, 0.64, 0.91, 0.78, 0.74, 0.93, 0.73, 0.56]),
 array([0.87, 0.94, 0.83, 0.65, 0.91, 0.78, 0.74, 0.92, 0.73, 0.56]))

### Time

In [7]:
metric = "Time"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

round_mean_at = 2
round_std_at = 3

for dataset_name in dataset_names:
    dataset_models = list(results_perf[dataset_name][0].keys())
    use_df = pd.DataFrame([pd.DataFrame(results_perf[dataset_name][fold_num]).loc[metric,models] for fold_num in results_perf[dataset_name].keys()],index=results_perf[dataset_name].keys())/60
    
    df_mean = pd.DataFrame(use_df.mean(axis=0).round(round_mean_at).astype(str) + " (" + use_df.std(axis=0).round(round_std_at).astype(str) + ")").transpose()
    model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}
    dataset_res_dict[dataset_name] = model_dict
    
    best_models[dataset_name] = use_df.columns[use_df.mean(axis=0).argmin()]

    t_test_res = np.array([stats.ttest_rel(use_df[best_models[dataset_name]].values, use_df[model].values)[1] if model in dataset_models else 0 for model in models]).round(3)
    t_test_res[np.isnan(t_test_res)] = 1.
    t_test_results[dataset_name] = t_test_res
    
res_df = pd.DataFrame(dataset_res_dict).transpose()
    
def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_results[dataset_name][i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

# res_df.style.apply(negative_bold)
res_df

,GMENN0.1,GMENN0.01,GMENN0.001
churn,2.15 (0.148),3.04 (0.291),15.28 (2.65)
kdd_internet_usage,2.1 (0.627),3.42 (0.472),15.25 (2.776)
Amazon_employee_access,0.88 (0.072),6.23 (1.233),18.71 (6.388)
Click_prediction_small,2.4 (0.622),11.42 (3.523),33.15 (5.643)
adult,1.08 (0.186),1.2 (0.079),3.62 (0.215)
KDDCup09_upselling,1.75 (0.46),11.17 (1.099),53.67 (13.684)
kick,1.38 (0.183),1.98 (0.422),6.59 (0.784)
open_payments,1.08 (0.067),5.64 (2.212),10.32 (3.626)
road-safety-drivers-sex,4.31 (0.98),5.7 (1.821),16.0 (1.339)
porto-seguro,3.88 (0.577),2.42 (0.072),4.56 (0.26)


In [8]:
res_df["GMENN0.1"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN0.01"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN0.001"].apply(lambda x: float(x.split(" ")[0])).values

(array([2.15, 2.1 , 0.88, 2.4 , 1.08, 1.75, 1.38, 1.08, 4.31, 3.88]),
 array([ 3.04,  3.42,  6.23, 11.42,  1.2 , 11.17,  1.98,  5.64,  5.7 ,
         2.42]),
 array([15.28, 15.25, 18.71, 33.15,  3.62, 53.67,  6.59, 10.32, 16.  ,
         4.56]))